# 生长介质 (Growth media)

营养物的可用性对代谢物通量影响很大，并且 `cobrapy` 提供一些帮助函数来管理外部环境和代谢模型的交换. 在实验设置中 "环境" 经常由生长介质构成, 因此是所有对模型有机物可用的代谢物和同因子 (co-factors) 的浓度 (concentrations). 但是, 基于约束的代谢模型只考虑通量. 所以, 你无法简单使用浓度，因为通量有单位 `mmol / [gDW h]` (concentration per gram dry weight of cells and hour). 

并且, 你在设置输入的通量的上界，而非通量本身. 有一些粗略的估计. 比如说, 如果你每 24h 提供 1 mol 葡萄糖给 1 gram 细菌，你可能想设置葡萄糖交换通量的上界为 `1 mol / [1 gDW * 24 h]`，既然这是能够输入的名义 (nominal) 上的最大值. 你无法保证葡萄糖会以那个通量吸收. 因此, 建议的交换通量的数据是直接通量测量而来的——比如说通过时间过程检验-代谢物测量 (timecourse exa-metabolome measurements) 获得的。

所以在 COBRApy 中怎么做? 模型的当前生长介质值由 `medium` 属性管理.  

In [1]:
from cobra.test import create_test_model

model = create_test_model("textbook")
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

这会返回一个包含所有活跃交换通量（拥有非 0 通量界限的那些）上界的字典. 现在我们看到我们已经启用了有氧生长 (aerobic growth). 你可以更改模型的生长介质，通过分配一个字典到 `model.medium`，这个字典把交换反应和它们对应的输入界限做了一一映射关系. 现在让我们关闭氧气输入，强迫厌氧生长.

In [2]:
medium = model.medium
medium["EX_o2_e"] = 0.0
model.medium = medium

model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

现在我们可以看到氧气的输入被移除出活跃交换，可以验证生长速率降低了。

In [3]:
model.slim_optimize()

0.21166294973530736

这里有个小陷阱. `model.medium` 无法被直接赋值. 这样是不行的:

In [4]:
model.medium["EX_co2_e"] = 0.0
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

可以看到 `EX_co2_e` 设为了非 0 值. 这是因为 model.medium 只是当前交换通量的一个副本. 直接像这样赋值 `model.medium[...] = ...` **不会** 改变模型. 你只能赋值改变了上限的整个字典:

In [5]:
medium = model.medium
medium["EX_co2_e"] = 0.0
model.medium = medium

model.medium  # now it worked

{'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

设置生长介质与上下文管理器也有关，所以你可以用可逆的方式设置指定的生长介质。

In [6]:
model = create_test_model("textbook")

with model:
    medium = model.medium
    medium["EX_o2_e"] = 0.0
    model.medium = medium
    print(model.slim_optimize())
print(model.slim_optimize())
model.medium

0.21166294973530736
0.8739215069684102


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

现在介质的改变只会在 `with` 块中生效，并会自动回退.

## 最小介质

有时候你可能会对维持指定生长速度的最小生长介质感兴趣，这被称为「最小介质」。这里我们提供了函数 `minimal_medium`，它默认包含了含最低总输入通量的介质。这个函数需要两个参数：模型本身和模型需要达到了最小生长速率（或者其他目标）

In [7]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

EX_glc__D_e    10.000000
EX_nh4_e        4.765319
EX_o2_e        21.799493
EX_pi_e         3.214895
dtype: float64

所以我们可以看到生长实际上被葡萄糖输入所限制。

另外你可能会对有最小数量活跃输入的最小介质感兴趣. 这可以通过使用 `minimize_components` 参数来实现 (注意这使用了 MIP（混合整数规划） 表示，所以会很慢).

In [8]:
minimal_medium(model, 0.1, minimize_components=True)

EX_glc__D_e    10.000000
EX_nh4_e        1.042503
EX_pi_e         0.703318
dtype: float64

当最小化输入通量时，可能会有许多额外的解。为了获得多个，你可以传一个正数给 `minimize_components`，这会给你最多那么多额外的解。让我们试试我们的模型，并且使用 `open_exchanges` 参数，这会给所有的输入反应分配一个大的上界。返回值是 `pandas.DataFrame`。

In [9]:
minimal_medium(model, 0.8, minimize_components=8, open_exchanges=True)

0           1            2           3
EX_fru_e       0.000000  521.357767     0.000000    0.000000
EX_glc__D_e    0.000000    0.000000     0.000000  519.750758
EX_gln__L_e    0.000000   40.698058    18.848678    0.000000
EX_glu__L_e  348.101944    0.000000     0.000000    0.000000
EX_mal__L_e    0.000000    0.000000  1000.000000    0.000000
EX_nh4_e       0.000000    0.000000     0.000000   81.026921
EX_o2_e      500.000000    0.000000     0.000000    0.000000
EX_pi_e       66.431529   54.913419    12.583458   54.664344

所以总共有 4 个额外解。一个有氧的，和三个用到了不同碳来源的厌氧的解。

## 界限反应 (Boundary reactions)

除了交换反应以外，还有一些其他类型的界限反应，比如说需求或者缩水反应 (demand or sink reactions)。`cobrapy` 使用多个启发式算法来识别它们，它们可以使用恰当的属性访问。

对于交换反应:

In [10]:
ecoli = create_test_model("ecoli")
ecoli.exchanges[0:5]

[<Reaction EX_12ppd__R_e at 0x131b4a58d0>,
 <Reaction EX_12ppd__S_e at 0x131b471c50>,
 <Reaction EX_14glucan_e at 0x131b471e10>,
 <Reaction EX_15dap_e at 0x131b471e48>,
 <Reaction EX_23camp_e at 0x131b471f98>]

对于需求反应:

In [11]:
ecoli.demands

[<Reaction DM_4CRSOL at 0x131b3162b0>,
 <Reaction DM_5DRIB at 0x131b4712e8>,
 <Reaction DM_AACALD at 0x131b471400>,
 <Reaction DM_AMOB at 0x131b4714e0>,
 <Reaction DM_MTHTHF at 0x131b4715f8>,
 <Reaction DM_OXAM at 0x131b4716d8>]

对于缩水反应:

In [12]:
ecoli.sinks

[]

所有界限反应 (任何在系统中消耗/提供质量的反应) 可以用 `boundary` 属性获得:

In [13]:
ecoli.boundary[0:10]

[<Reaction DM_4CRSOL at 0x131b3162b0>,
 <Reaction DM_5DRIB at 0x131b4712e8>,
 <Reaction DM_AACALD at 0x131b471400>,
 <Reaction DM_AMOB at 0x131b4714e0>,
 <Reaction DM_MTHTHF at 0x131b4715f8>,
 <Reaction DM_OXAM at 0x131b4716d8>,
 <Reaction EX_12ppd__R_e at 0x131b4a58d0>,
 <Reaction EX_12ppd__S_e at 0x131b471c50>,
 <Reaction EX_14glucan_e at 0x131b471e10>,
 <Reaction EX_15dap_e at 0x131b471e48>]